In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# want to investigate time series relations of StackOverflow questions regarding Python over time


# Load time series data
stack_overfl_qs = pd.read_csv('StackOverflowData.csv')
stack_overfl_qs.head()



,month,nltk,spacy,stanford-nlp,python,r,numpy,scipy,matlab,machine-learning,...,Plato,Sympy,Flair,stanford-nlp.1,pyqt,Nolearn,Lasagne,OCR,Apache-spark-mlib,azure-virtual-machine
0,09-Jan,0,0,0,631,8,6,2,19,8,...,0,1,0,0,5,0,0,5,0,0
1,09-Feb,1,0,0,633,9,7,3,27,4,...,0,0,0,0,5,0,0,11,0,0
2,09-Mar,0,0,0,766,4,4,2,24,3,...,0,0,0,0,7,0,0,2,0,0
3,09-Apr,0,0,0,768,12,6,3,32,10,...,0,0,0,0,11,0,0,5,0,0
4,09-May,1,0,0,1003,2,7,2,42,7,...,0,0,0,0,10,0,0,3,0,0


In [19]:
stack_overfl_qs['month'] = pd.to_datetime(stack_overfl_qs['month'], format="%y %B")
print(stack_overfl_qs.head())
timeseries = stack_overfl_qs[['month', 'python']]

timeseries.set_index('month', inplace = True)
timeseries = timeseries.dropna()

timeseries.head()

       month  nltk  spacy  stanford-nlp  python   r  numpy  scipy  matlab  \
0 1900-01-09     0      0             0     631   8      6      2      19   
1 1900-02-09     1      0             0     633   9      7      3      27   
2 1900-03-09     0      0             0     766   4      4      2      24   
3 1900-04-09     0      0             0     768  12      6      3      32   
4 1900-05-09     1      0             0    1003   2      7      2      42   

   machine-learning  ...  Plato  Sympy  Flair  stanford-nlp.1  pyqt  Nolearn  \
0                 8  ...      0      1      0               0     5        0   
1                 4  ...      0      0      0               0     5        0   
2                 3  ...      0      0      0               0     7        0   
3                10  ...      0      0      0               0    11        0   
4                 7  ...      0      0      0               0    10        0   

   Lasagne  OCR  Apache-spark-mlib  azure-virtual-machin

,python
month,
1900-01-09,631
1900-02-09,633
1900-03-09,766
1900-04-09,768
1900-05-09,1003
